In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
driver = webdriver.Chrome()

In [ ]:
driver.get('data')

In [ ]:
driver.title

In [ ]:
email = driver.find_element(By.ID, 'username')
email.send_keys(os.getenv('EMAIL'))

In [ ]:
password = driver.find_element(By.ID, 'password')
password.send_keys(os.getenv('PASSWORD'))

In [ ]:
password.submit()

In [ ]:
url = "data"

driver.get(url)

In [ ]:
page_source = driver.page_source

In [ ]:
soup = BeautifulSoup(page_source, 'lxml')

In [ ]:
# soup.get_text()

In [ ]:
profile = soup.find('main', {'class': 'scaffold-layout__main'})

In [ ]:
# print(profile.get_text())

In [ ]:
sections = profile.find_all('section', {'class': 'artdeco-card'})

In [ ]:
len(sections)

In [ ]:
# sections

In [ ]:
sections_text = [section.get_text() for section in sections]

In [ ]:
import re

def clean_text(text):
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'\t+', '\t', text)
    text = re.sub(r'\t\s+', ' ', text) 
    text = re.sub(r'\n\s+', '\n', text)
    return text

In [ ]:
sections_text = [clean_text(section) for section in sections_text]

In [ ]:
def remove_duplicates(text):
    lines = text.split('\n')
    new_lines = []
    for line in lines:
        if line[:len(line)//2] == line[len(line)//2:]:
            new_lines.append(line[:len(line)//2])
        else:
            new_lines.append(line)

    return '\n'.join(new_lines)

In [ ]:
sections_text = [remove_duplicates(section) for section in sections_text]

In [ ]:
print(sections_text[1])

In [ ]:
sections_text

In [ ]:
from langchain_ollama import ChatOllama

from langchain_core.prompts import (SystemMessagePromptTemplate, 
                                    HumanMessagePromptTemplate,
                                    ChatPromptTemplate)



from langchain_core.output_parsers import StrOutputParser

base_url = "http://localhost:11434"
# model = 'data'
model = "data"

llm = ChatOllama(base_url=base_url, model=model)


system = SystemMessagePromptTemplate.from_template("""You are helpful AI assistant who answer data profile parsing related 
                                                    user question based on the provided profile text data.""")

def ask_llm(prompt):
    prompt = HumanMessagePromptTemplate.from_template(prompt)

    messages = [system, prompt]
    template = ChatPromptTemplate(messages)

    qna_chain = template | llm | StrOutputParser()

    return qna_chain.invoke({})

In [ ]:
llm

In [ ]:
ask_llm("hello")

In [ ]:
template = """
Extract and return the requested information from the data profile data in a concise, point-by-point format (up to 5 points). Avoid preambles or any additional context.

### data Profile Data:
{}

### Information to Extract:
Extract '{}' in bullet points, limiting the output to 5 points. Provide only the necessary details.
Remember, It is data profile data.

### Extracted Data:"""

context = sections_text[0]
k = "Name and Headline"

prompt = template.format(context, k)
response = ask_llm(prompt)

In [ ]:
print(response)

In [ ]:
section_keys = ['Name and Headline']
for section in sections_text[1:]:
    # print(section.strip().split('\n'))
    section_keys.append(section.strip().split('\n')[0])

section_keys

In [ ]:
responses = {}

for k,context in zip(section_keys, sections_text):
    prompt = template.format(context, k)
    response = ask_llm(prompt=prompt)
    responses[k] = response

In [ ]:
print(responses)

In [ ]:
import json

with open('data', 'w') as f:
    json.dump(responses, f, indent=4)

In [ ]:
template = """You are provided with LinkedIn profile data in JSON format.
            Parse the data according to the specified schema, correct any spelling errors,
            and condense the information if possible.

### data Profile JSON Data:
{context}

### Schema You need to follow:
You need to extract
Name:
Headline:
About:
Experience:
Education:
Skills:
Projects:
Summary:

Do not return preambles or any other information.
### Parsed Data:"""

prompt = template.format(context=responses).replace("{", "{{").replace("}", "}}") 
response = ask_llm(prompt=prompt)
# response

In [ ]:
print(response)